# The Canonical Time-series Characteristics (catch22) transform

catch22\[1\] is a collection of 22 time series features extracted from the 7000+ present in the _hctsa_\[2\]\[3\] toolbox.
A hierarchical clustering was performed on the correlation matrix of features that performed better than random chance to remove redundancy.
These clusters were sorted by balanced accuracy using a decision tree classifier and a single feature was selected from the 22 clusters formed, taking into account balanced accuracy results, computational efficiency and interpretability.

In this notebook, we will demonstrate how to use the catch22 transformer on the ItalyPowerDemand univariate and BasicMotions multivariate datasets. We also show catch22 used for classification with a random forest classifier.

#### References:

\[1\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[2\] Fulcher, B. D., & Jones, N. S. (2017). hctsa: A computational framework for automated time-series phenotyping using massive feature extraction. Cell systems, 5(5), 527-531.

\[3\] Fulcher, B. D., Little, M. A., & Jones, N. S. (2013). Highly comparative time-series analysis: the empirical structure of time series and their methods. Journal of the Royal Society Interface, 10(83), 20130048.

## 1. Imports

In [1]:
from sklearn import metrics

from sktime.classification.feature_based import Catch22Classifier
from sktime.datasets import load_basic_motions, load_italy_power_demand
from sktime.transformations.panel.catch22 import Catch22

## 2. Load data

In [2]:
IPD_X_train, IPD_y_train = load_italy_power_demand(split="train", return_X_y=True)
IPD_X_test, IPD_y_test = load_italy_power_demand(split="test", return_X_y=True)
IPD_X_test = IPD_X_test[:50]
IPD_y_test = IPD_y_test[:50]

print(IPD_X_train.shape, IPD_y_train.shape, IPD_X_test.shape, IPD_y_test.shape)

BM_X_train, BM_y_train = load_basic_motions(split="train", return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split="test", return_X_y=True)

print(BM_X_train.shape, BM_y_train.shape, BM_X_test.shape, BM_y_test.shape)

(67, 1) (67,) (50, 1) (50,)
(40, 6) (40,) (40, 6) (40,)


## 3. catch22 transform

### Univariate

The catch22 features are provided in the form of a transformer, `Catch22`.
From this the transformed data can be used for a variety of time series analysis tasks.

In [3]:
c22_uv = Catch22()
c22_uv.fit(IPD_X_train, IPD_y_train)

Catch22()

In [4]:
transformed_data_uv = c22_uv.transform(IPD_X_train)
print(transformed_data_uv.head())

        0         1     2         3         4    5     6         7         8   \
0  1.15865 -0.217225   8.0  0.291667 -0.625000  3.0   6.0  0.468048  0.589049   
1  0.91817 -0.214750  15.0  0.208333 -0.666667  4.0   8.0  0.702776  0.196350   
2 -0.27318 -0.085860   4.0  0.875000  0.250000  2.0   5.0  0.310569  0.589049   
3  0.04840 -0.450100  13.0  0.166667 -0.625000  4.0  10.0  0.804061  0.196350   
4  0.42639 -0.450720  16.0  0.291667 -0.666667  4.0   7.0  0.675478  0.196350   

         9   ...   12        13   14        15        16        17  18  19  \
0  0.836757  ...  3.0  1.000000  5.0  1.771467  0.750000  0.240589 NaN NaN   
1  0.666157  ...  4.0  0.869565  5.0  1.665694  0.500000  0.388200 NaN NaN   
2  0.865062  ...  2.0  0.913043  5.0  1.722958  0.666667  0.089110 NaN NaN   
3  0.648316  ...  4.0  0.869565  6.0  1.722958  0.666667  0.332417 NaN NaN   
4  0.657948  ...  4.0  0.913043  6.0  1.722958  0.500000  0.318433 NaN NaN   

         20   21  
0  0.040000  0.0  
1  0.1

The transform `Catch22` method will process all 22 features.
For individual features, the transform_single_feature method can be used when provided with a numeric feature ID or the feature name.

In [5]:
transformed_feature_uv = c22_uv._transform_single_feature(IPD_X_train, "CO_f1ecac")
print(transformed_feature_uv)

[3 4 2 4 4 3 3 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 3 4 4 3 4 4
 3 4 4 3 4 4 3 4 4 3 4 3 3 4 3 4 3 3 4 4 4 3 2 4 4 4 4 4 4 2]


### Multivariate

Transformation of multivariate data is supported by `Catch22`.
The default procedure will concatenate each column prior to transformation.

In [6]:
c22_mv = Catch22()
c22_mv.fit(BM_X_train, BM_y_train)

Catch22()

In [7]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

         0         1     2         3         4    5    6         7         8   \
0 -0.208846  0.136909   9.0 -0.510833 -0.421667  2.0  5.0  0.156349  0.632000   
1  0.739665  0.144455  79.0 -0.520000 -0.318333  2.0  6.0  0.363428  0.570641   
2  0.008406  0.199387  25.0 -0.446667 -0.471667  3.0  6.0  0.225461  0.533825   
3  0.090501 -0.033292  17.0 -0.530000 -0.353333  2.0  5.0  0.045657  0.625864   
4 -0.096920 -0.009260  26.0 -0.513333 -0.685833  3.0  5.0  0.041056  0.570641   

         9   ...   12        13   14        15        16        17        18  \
0  0.624990  ...  2.0  0.701169  7.0  1.956105  0.666667  0.183226  0.659574   
1  0.765986  ...  3.0  0.562604  7.0  1.862992  0.250000  0.083613  0.872340   
2  0.523145  ...  2.0  0.754591  8.0  1.923265  1.000000  0.370307  0.659574   
3  0.317573  ...  2.0  0.736227  7.0  2.012542  0.666667  0.679593  0.638298   
4  0.220888  ...  2.0  0.589316  7.0  1.869332  0.750000  0.601701  0.617021   

         19        20    21  
0 

## 4. catch22 Forest Classifier

For classification tasks the default classifier to use with the catch22 features is random forest classifier.
An implementation making use of the `RandomForestClassifier` from sklearn built on catch22 features is provided in the form on the `Catch22Classifier` for ease of use.

In [8]:
c22f = Catch22Classifier(random_state=0)
c22f.fit(IPD_X_train, IPD_y_train)

Catch22Classifier(estimator=RandomForestClassifier(n_estimators=200, n_jobs=1),
                  random_state=0)

In [9]:
c22f_preds = c22f.predict(IPD_X_test)
print("C22F Accuracy: " + str(metrics.accuracy_score(IPD_y_test, c22f_preds)))

C22F Accuracy: 0.9
